# Importing Libraries

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Reading Data

In [23]:
train = pd.read_csv(r"D:\Data Science Course\Projects\Analytics vidya\Cross Sales Prediction\train.csv")
test = pd.read_csv(r"D:\Data Science Course\Projects\Analytics vidya\Cross Sales Prediction\test.csv")
sample_sub = pd.read_csv(r"D:\Data Science Course\Projects\Analytics vidya\Cross Sales Prediction\sample_submission.csv")

# Label Encoding Categorical Columns

In [24]:
train['Vehicle_Age']=train['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})
train['Gender']=train['Gender'].replace({'Male':1,'Female':0})
train['Vehicle_Damage']=train['Vehicle_Damage'].replace({'Yes':1,'No':0})

test['Vehicle_Age']=test['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})
test['Gender']=test['Gender'].replace({'Male':1,'Female':0})
test['Vehicle_Damage']=test['Vehicle_Damage'].replace({'Yes':1,'No':0})

---
# Model Building

In [25]:
features=['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

In [26]:
cat_col=['Gender','Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage','Policy_Sales_Channel']

In [27]:
train['Region_Code']=train['Region_Code'].astype(int)
test['Region_Code']=test['Region_Code'].astype(int)
train['Policy_Sales_Channel']=train['Policy_Sales_Channel'].astype(int)
test['Policy_Sales_Channel']=test['Policy_Sales_Channel'].astype(int)

In [28]:
X=train[features]
y=train['Response']

In [29]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=150303,stratify=y,shuffle=True)

---
# Ensembling

In [51]:
# Cat Model Building
model = CatBoostClassifier()
model= model.fit(X_train, y_train,cat_features=cat_col,eval_set=(X_test, y_test),early_stopping_rounds=30,verbose=100)
y_cat = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, proba)

Learning rate set to 0.128106
0:	learn: 0.4910032	test: 0.4914047	best: 0.4914047 (0)	total: 159ms	remaining: 2m 38s
100:	learn: 0.2632019	test: 0.2644836	best: 0.2644836 (100)	total: 11.6s	remaining: 1m 42s
200:	learn: 0.2614128	test: 0.2643219	best: 0.2643075 (176)	total: 23.8s	remaining: 1m 34s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.2642942315
bestIteration = 211

Shrink model to first 212 iterations.


0.8586121579979858

In [58]:
# LGBM Tuned Model
LGBM = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.20980938655996545,
               max_depth=3, min_child_samples=143, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=283, n_jobs=-1, num_leaves=3668,
               objective=None, random_state=0, reg_alpha=2, reg_lambda=5,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
LGBM = LGBM.fit(train[features], train['Response'])
proba = LGBM.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, proba)

0.862845643896715

In [53]:
LGBM_pred = LGBM.predict_proba(test[features])[:, 1]
cat_pred = model.predict_proba(test[features])[:, 1]

In [54]:
final = ((LGBM_pred * 0.45) + (cat_pred * 0.55))/2

In [55]:
sample_sub['Response'] = final

In [56]:
sample_sub.to_csv("D:\\Data Science Course\\Projects\\Analytics vidya\\Cross Sales Prediction\\Tuned\\Catboost.csv", index = False)

---